# PRODUCER

# Análisis de Transacciones en la Plataforma Uber

## Universidad Tecnológica de Panamá  
### Facultad de Ingeniería de Sistemas Computacionales  
### Maestría en Analítica de Datos  

**Integrantes:**
- Ávila, Isaac  
- Sanjur, Andy

### 1. Descripción del caso simulado

En la última década, las plataformas de solicitud de viajes han adquirido una relevancia significativa en el ámbito de la movilidad urbana. Gracias a su practicidad, cada vez más usuarios y conductores optan por este tipo de soluciones de transporte privado. En una ciudad tan grande como Nueva York (NYC), pueden registrarse cientos de viajes por minuto, lo cual representa un reto importante para el análisis de datos en tiempo real mediante métodos convencionales, o bien vuelve ineficiente el análisis de datos históricos para el ajuste dinámico de tarifas según la demanda del momento.

El siguiente caso simulado presenta una serie de transacciones realizadas por la plataforma Uber. Cada registro corresponde a un viaje efectuado desde un punto A hasta un punto B, incluyendo su respectivo detalle. A partir de estos datos, se busca analizar los siguientes aspectos:

- ¿Cuáles son los puntos de partida más comunes?
- ¿Cuáles son los destinos más concurridos?
- ¿En qué días y a qué horas existe mayor frecuencia de solicitudes de viaje?
- Determinar cuáles son las matrículas con mayor cantidad de demoras.
- Determinar los minutos promedio de demora.

Para el análisis se utiliza un conjunto de datos en formato `.csv`, recopilado de la plataforma Kaggle:  
https://www.kaggle.com/datasets/ahmedramadan74/uber-nyc  

Este conjunto de datos incluye los siguientes campos relevantes para el caso de uso:

- **index_trip (int):** Número único del evento.
- **hvfhs_license_num (str):** Tipo de servicio contratado.
- **request_datetime (timestamp):** Fecha y hora en que se solicita el servicio.
- **pickup_datetime (timestamp):** Fecha y hora de inicio del viaje.
- **dropoff_datetime (timestamp):** Fecha y hora de finalización del viaje.
- **PULocationID (int):** Zona donde inicia el viaje.
- **DOLocationID (int):** Zona donde finaliza el viaje.
- **trip_miles (double):** Distancia total del viaje.
- **trip_time (int):** Duración del viaje en segundos.
- **base_passenger_fare (double):** Tarifa base del viaje.
- **tolls (double):** Total de peajes.
- **sales_tax (double):** Total de impuestos.
- **congestion_surcharge (double):** Costos adicionales por congestión vehicular.
- **airport_fee (double):** Tarifa adicional por aeropuerto (USD 2.50).
- **tips (double):** Propinas.
- **driver_pay (double):** Pago al conductor.
- **hour (double):** Hora de la solicitud.
- **year (double):** Año de la solicitud.
- **month (double):** Mes de la solicitud.
- **day (double):** Día de la solicitud.
- **on_time_pickup (int):** Indicador de llegada a tiempo.

---

### 2. Justificación del uso de Big Data en el problema

La alta demanda y el gran volumen de información que puede generar la aplicación durante las horas pico dificultan el análisis en tiempo real mediante herramientas tradicionales de analítica de datos. En este contexto, las oportunidades de negocio pueden perder relevancia si la información no es procesada y utilizada de manera inmediata.

Para este caso, con el objetivo de mantener la rapidez en el análisis y manejar eficientemente el volumen de datos que se generan cada minuto, se emplean tecnologías orientadas a Big Data, tales como **Apache Cassandra** para el almacenamiento distribuido de datos, **Apache Kafka** para la transmisión de datos en tiempo real (streaming) y **PySpark** para la transformación, consulta y exportación de la información analizada.

### 3. Proceso utilizado para el análisis

#### 3.1 Creación de la sesión de Spark

En esta etapa se importa la librería **SparkSession** y se crea la sesión de Apache Spark.  
Para ello, se configura el **host** y el **puerto** de conexión que permitirán establecer la comunicación con el entorno de procesamiento distribuido.

##### 3.1.1 Definición de constantes de configuración

Con el fin de mejorar la mantenibilidad y portabilidad del código, se definen constantes que agrupan los parámetros de configuración de la sesión de Spark y la conexión con la base de datos Apache Cassandra. Este enfoque permite modificar fácilmente los valores de conexión sin afectar la lógica del procesamiento del análisis.


### 1. Definición de constantes de configuración

Con el fin de mejorar la mantenibilidad y portabilidad del código, se definen constantes que agrupan los parámetros de configuración de la sesión de Spark y la conexión con la base de datos Apache Cassandra. Este enfoque permite modificar fácilmente los valores de conexión sin afectar la lógica del procesamiento del análisis.


In [1]:
import os
from cassandra.cluster import Cluster
from cassandra.policies import RoundRobinPolicy
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

# =======================================================
# 1. CONSTANTES DE CONEXIÓN Y CONFIGURACIÓN
# =======================================================

# Hosts de los nodos de Cassandra
CASSANDRA_HOST_ANDY = "localhost"
CASSANDRA_PORT = "9042"

# Nombre de la aplicación Spark
APP_NAME = "Proyecto_Final_Uber_Trips"

# Conector de Spark-Cassandra
CASSANDRA_CONNECTOR = "com.datastax.spark:spark-cassandra-connector_2.13:3.5.1"

# Configuración del Keyspace y la Tabla en Cassandra
KEYSPACE = "trips"
TABLE_RAW = "uber_trips"
TABLE_CLEAN = "uber_trips_clean"
TABLE_PROD = "uber_trips_prod"
REPLICATION_FACTOR = 1
BORRAR_DATOS = False
# Si el error persiste, dejar en True para borrar y recrear la tabla con el esquema correcto (minúsculas).
FORZAR_RECREACION_ESQUEMA = True

# Definición de las 33 columnas que deben existir en la tabla final de Cassandra, en minúsculas.
COLUMNAS_FINALES_CASSANDRA = [
    "index_trip", "hvfhs_license_num", "dispatching_base_num", "originating_base_num",
    "request_datetime", "on_scene_datetime", "pickup_datetime", "dropoff_datetime",
    "pulocationid", "dolocationid", "trip_miles", "trip_time", "base_passenger_fare",
    "tolls", "bcf", "sales_tax", "congestion_surcharge", "airport_fee", "tips",
    "driver_pay", "shared_request_flag", "shared_match_flag", "access_a_ride_flag",
    "wav_request_flag", "wav_match_flag", "hour", "year", "month", "day",
    "on_time_pickup",
    "uber_sales",
    "pickup_zone",
    "delivery_zone"
]

#### 2 Creación de la sesión de Spark

En esta etapa se importa la librería **SparkSession** y se crea la sesión de Apache Spark.  
Para ello, se configura el **host** y el **puerto** de conexión que permitirán establecer la comunicación con el entorno de procesamiento distribuido.

In [2]:
# =======================================================
# 2. INICIALIZACIÓN DE SPARK
# =======================================================

spark = SparkSession.builder \
    .appName(APP_NAME) \
    .config("spark.jars.packages", CASSANDRA_CONNECTOR) \
    .config("spark.cassandra.connection.host", CASSANDRA_HOST_ANDY) \
    .config("spark.cassandra.connection.port", CASSANDRA_PORT) \
    .getOrCreate()

print("SparkSession iniciada y conectada a Cassandra.")


:: loading settings :: url = jar:file:/home/admin/jupyter_venv/lib/python3.12/site-packages/pyspark/jars/ivy-2.5.3.jar!/org/apache/ivy/core/settings/ivysettings.xml
Ivy Default Cache set to: /home/admin/.ivy2.5.2/cache
The jars for the packages stored in: /home/admin/.ivy2.5.2/jars
com.datastax.spark#spark-cassandra-connector_2.13 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-4babbce5-578f-48ba-b4ba-55fcc627fde6;1.0
	confs: [default]
	found com.datastax.spark#spark-cassandra-connector_2.13;3.5.1 in central
	found com.datastax.spark#spark-cassandra-connector-driver_2.13;3.5.1 in central
	found org.scala-lang.modules#scala-collection-compat_2.13;2.11.0 in central
	found org.scala-lang.modules#scala-parallel-collections_2.13;1.0.4 in central
	found org.apache.cassandra#java-driver-core-shaded;4.18.1 in central
	found com.datastax.oss#native-protocol;1.5.1 in central
	found com.datastax.oss#java-driver-shaded-guava;25.1-jre-graal-sub-1 in central
	

SparkSession iniciada y conectada a Cassandra.


### 3 Integración con Cassandra

En esta etapa se realiza la creación y conexión al clúster de **Apache Cassandra**, el cual es utilizado como sistema de almacenamiento distribuido. Se define el **keyspace** denominado `trips` y se crea la tabla `uber_trips`, estructurada de acuerdo con las columnas del conjunto de datos analizado. Finalmente, se efectúa la escritura de la información en Cassandra, persistiendo los datos previamente cargados y procesados mediante Apache Spark.


In [3]:
# =======================================================
# 3. CONEXIÓN Y DEFINICIÓN DE ESQUEMAS DE CASSANDRA
# =======================================================


cluster = Cluster(
    contact_points=[CASSANDRA_HOST_ANDY],
    port=int(CASSANDRA_PORT),
    load_balancing_policy=RoundRobinPolicy()
)

session = cluster.connect()
print("Conexión con Cassandra establecida.")

# Crear Keyspace (si no existe)
session.execute(f"""
CREATE KEYSPACE IF NOT EXISTS {KEYSPACE}
WITH replication = {{
    'class': 'SimpleStrategy',
    'replication_factor': {REPLICATION_FACTOR}
}};
""")
session.set_keyspace(KEYSPACE)
print(f"Keyspace '{KEYSPACE}' verificado/creado.")


# **PASO CRÍTICO DE LIMPIEZA DE ESQUEMA**
if FORZAR_RECREACION_ESQUEMA:
    print("\nADVERTENCIA: FORZANDO RECREACIÓN DE ESQUEMAS PARA CORREGIR PROBLEMAS DE CASE SENSITIVITY.")
    session.execute(f"DROP TABLE IF EXISTS {TABLE_RAW};")
    session.execute(f"DROP TABLE IF EXISTS {TABLE_CLEAN};")
    session.execute(f"DROP TABLE IF EXISTS {TABLE_PROD};")
    print("Tablas eliminadas. Procediendo a recrear.")


# 3.1. Creación de la tabla de datos crudos (Raw Table: uber_trips)
RAW_TABLE_SCHEMA = """
    index_trip int PRIMARY KEY, hvfhs_license_num text, dispatching_base_num text,
    originating_base_num text, request_datetime timestamp, on_scene_datetime timestamp,
    pickup_datetime timestamp, dropoff_datetime timestamp, pulocationid int, dolocationid int,
    trip_miles float, trip_time int, base_passenger_fare float, tolls float, bcf float,
    sales_tax float, congestion_surcharge float, airport_fee float, tips float, driver_pay float,
    shared_request_flag text, shared_match_flag text, access_a_ride_flag text,
    wav_request_flag text, wav_match_flag text, hour float, year float, month float, day float,
    on_time_pickup int
"""
session.execute(f"CREATE TABLE IF NOT EXISTS {TABLE_RAW} ({RAW_TABLE_SCHEMA});")
print(f"Tabla de datos crudos '{TABLE_RAW}' verificado/creada.")

# 3.2. Creación de la tabla de datos limpios (Clean Table: uber_trips_clean)
CLEAN_TABLE_SCHEMA = RAW_TABLE_SCHEMA + ", uber_sales float, pickup_zone text, delivery_zone text"
session.execute(f"CREATE TABLE IF NOT EXISTS {TABLE_CLEAN} ({CLEAN_TABLE_SCHEMA});")
print(f"Tabla de datos limpios '{TABLE_CLEAN}' verificado/creada.")

# 3.3. Creación de la tabla de datos de producción (Prod Table: uber_trips_prod)
session.execute(f"CREATE TABLE IF NOT EXISTS {TABLE_PROD} ({CLEAN_TABLE_SCHEMA});")
print(f"Tabla de producción '{TABLE_PROD}' verificado/creada.")

if FORZAR_RECREACION_ESQUEMA:
    print("\nIMPORTANTE: Una vez que el proceso sea exitoso, cambia 'FORZAR_RECREACION_ESQUEMA = False'.")

# 3.4. TRUNCATE de Tablas (Operación Desactivada)
if BORRAR_DATOS:
    pass
else:
    print("Borrado de datos (TRUNCATE) deshabilitado. Se procederá a DEPOSITAR los datos (APPEND).")

Conexión con Cassandra establecida.
Keyspace 'trips' verificado/creado.

ADVERTENCIA: FORZANDO RECREACIÓN DE ESQUEMAS PARA CORREGIR PROBLEMAS DE CASE SENSITIVITY.
Tablas eliminadas. Procediendo a recrear.
Tabla de datos crudos 'uber_trips' verificado/creada.
Tabla de datos limpios 'uber_trips_clean' verificado/creada.
Tabla de producción 'uber_trips_prod' verificado/creada.

IMPORTANTE: Una vez que el proceso sea exitoso, cambia 'FORZAR_RECREACION_ESQUEMA = False'.
Borrado de datos (TRUNCATE) deshabilitado. Se procederá a DEPOSITAR los datos (APPEND).


#### 4. Lectura e importación del conjunto de datos

En esta etapa se realiza la lectura del conjunto de datos en formato `.csv` utilizando PySpark. Posteriormente, se visualizan las primeras cinco filas del conjunto de registros con el fin de validar la correcta carga de la información. Adicionalmente, se emplea la función `printSchema()` para inspeccionar la estructura del esquema y los tipos de datos asociados a cada columna.

In [4]:
# =======================================================
# 4. LECTURA Y CARGA INICIAL (CSV -> raw_trips)
# =======================================================

raw_trips = spark.read.csv(
    "data/sampling_trips.csv",
    header=True,
    inferSchema=True
).limit(1000)

print("\n--- ESQUEMA LEÍDO DEL CSV (raw_trips) ---")
raw_trips.printSchema() # Muestra las columnas con mayúsculas/camelCase
print("------------------------------------------")


# **CORRECCIÓN 1: Renombrar a minúsculas ANTES de la primera carga**
raw_trips = raw_trips.withColumnRenamed("PULocationID", "pulocationid") \
                     .withColumnRenamed("DOLocationID", "dolocationid")

print("Columnas PULocationID/DOLocationID renombradas a pulocationid/dolocationid.")

print("\n--- Iniciando DEPOSITO de datos iniciales a uber_trips (Modo APPEND) ---")
raw_trips.repartition(10) \
    .write \
    .format("org.apache.spark.sql.cassandra") \
    .mode("append") \
    .options(table=TABLE_RAW, keyspace=KEYSPACE) \
    .save()
print(f"Datos iniciales ({raw_trips.count()} filas) DEPOSITADOS en la tabla '{TABLE_RAW}'.")


--- ESQUEMA LEÍDO DEL CSV (raw_trips) ---
root
 |-- index_trip: integer (nullable = true)
 |-- hvfhs_license_num: string (nullable = true)
 |-- dispatching_base_num: string (nullable = true)
 |-- originating_base_num: string (nullable = true)
 |-- request_datetime: timestamp (nullable = true)
 |-- on_scene_datetime: timestamp (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- trip_miles: double (nullable = true)
 |-- trip_time: integer (nullable = true)
 |-- base_passenger_fare: double (nullable = true)
 |-- tolls: double (nullable = true)
 |-- bcf: double (nullable = true)
 |-- sales_tax: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- airport_fee: double (nullable = true)
 |-- tips: double (nullable = true)
 |-- driver_pay: double (nullable = true)
 |-- shared_request_flag: string (nullabl

25/12/15 23:56:29 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
                                                                                

Datos iniciales (1000 filas) DEPOSITADOS en la tabla 'uber_trips'.


#### 5. Consulta de la tabla en Cassandra

En esta etapa se valida la conexión con **Apache Cassandra** mediante la lectura de los datos almacenados desde Apache Spark. Se realiza la consulta de la tabla `uber_trips`, perteneciente al keyspace `trips`, y los resultados se cargan en un DataFrame denominado `df_raw`. Finalmente, se visualizan las primeras diez filas con el objetivo de verificar la correcta persistencia y recuperación de la información.

In [5]:
# =======================================================
# 5. LECTURA DE DATOS DESDE CASSANDRA (READ)
# =======================================================

df_raw = spark.read \
    .format("org.apache.spark.sql.cassandra") \
    .options(table=TABLE_RAW, keyspace=KEYSPACE) \
    .load()
    
filas_iniciales = df_raw.count()
print(f"Lectura de df_raw desde Cassandra exitosa. Filas leídas: {filas_iniciales}")

Lectura de df_raw desde Cassandra exitosa. Filas leídas: 1000


#### 6. Transformación del conjunto de datos

Se realiza la lectura del archivo `.csv` con la información de las ubicaciones y se aplican transformaciones de tipo `JOIN` entre tablas. Posteriormente, se ejecutan cálculos entre columnas para determinar la comisión de la plataforma y se generan agregaciones temporales. Finalmente, los datos procesados y agregados son exportados en formato `.csv`.

In [6]:
# =======================================================
# 6. TRANSFORMACIÓN, LIMPIEZA Y ENRIQUECIMIENTO
# =======================================================

print("\n--- Iniciando Proceso de Transformación y Limpieza ---")

# --- PASO CRÍTICO DE LIMPIEZA ---
# ⚠️ CORRECCIÓN: Evitamos .dropna() global. Filtramos solo por columnas clave.
# Aseguramos que la columna de tarifa base exista, ya que es la clave del cálculo de ventas.
df_clean = df_raw.filter(col("base_passenger_fare").isNotNull()) 
# También aseguramos que las IDs de ubicación existan
df_clean = df_clean.filter(col("pulocationid").isNotNull()) 

filas_despues_limpieza = df_clean.count()
print(f"DEBUG: Filas después de limpieza específica (base_fare, pu_id): {filas_despues_limpieza}")


# CÁLCULO uber_sales
df_clean = df_clean.withColumn(
    "uber_sales",
    col("base_passenger_fare") + col("tips") - col("driver_pay")
)

# LECTURA DE ZONAS (Lookup Table)
loc = spark.read \
    .csv("data/taxi_zone_lookup.csv", header=True, inferSchema=True) \
    .drop("Borough", "service_zone")

# Preparamos las tablas para el join 
loc_pickup = loc.withColumnRenamed("Zone", "pickup_zone").withColumnRenamed("LocationID", "PULocationID_lookup")
loc_dropoff = loc.withColumnRenamed("Zone", "delivery_zone").withColumnRenamed("LocationID", "DOLocationID_lookup")

# JOINS (Enriquecimiento de datos)
# 1. Join para obtener el nombre de la zona de recogida
df_clean = df_clean.join(
    loc_pickup,
    col("pulocationid") == col("PULocationID_lookup"),
    "left"
).drop(col("PULocationID_lookup")) 

# 2. Join para obtener el nombre de la zona de destino
df_clean = df_clean.join(
    loc_dropoff,
    col("dolocationid") == col("DOLocationID_lookup"),
    "left"
).drop(col("DOLocationID_lookup")) 

filas_despues_joins = df_clean.count()
print(f"DEBUG: Filas después de Joins (debería ser el mismo conteo): {filas_despues_joins}")


# AGREGACIÓN
df_agg_uber_sales = df_clean.groupBy(
    "year", "month", "day", "hour"
).sum("uber_sales").withColumnRenamed("sum(uber_sales)", "total_uber_sales")


# **PASO CRÍTICO 6.1: VALIDACIÓN Y SELECCIÓN FINAL DE COLUMNAS**
print("\n--- 6.1: Validación Final de Esquema para Cassandra ---")

# Filtrar el DataFrame para incluir SOLO las columnas de Cassandra
df_clean_final = df_clean.select(*COLUMNAS_FINALES_CASSANDRA)
df_clean = df_clean_final
print("Validación de columnas para Cassandra OK. Listo para depositar.")

# **CONTEO DE FILAS FINALES**
filas_depositadas = df_clean.count()



--- Iniciando Proceso de Transformación y Limpieza ---


DEBUG: Filas después de limpieza específica (base_fare, pu_id): 1000
DEBUG: Filas después de Joins (debería ser el mismo conteo): 1000

--- 6.1: Validación Final de Esquema para Cassandra ---
Validación de columnas para Cassandra OK. Listo para depositar.


In [7]:
# =======================================================
# 7. EXPORTACIÓN DE DATOS PROCESADOS A CASSANDRA
# =======================================================


# Carga a uber_trips_clean (DEPOSITO en modo APPEND)
print("\n--- DEPOSITANDO datos limpios a uber_trips_clean ---")
df_clean.write \
    .format("org.apache.spark.sql.cassandra") \
    .mode("append") \
    .options(table=TABLE_CLEAN, keyspace=KEYSPACE) \
    .save()

print(f"RESULTADO: {filas_depositadas} filas DEPOSITADAS en la tabla '{TABLE_CLEAN}'.")

# Carga a uber_trips_prod (DEPOSITO en modo APPEND)
print("--- DEPOSITANDO datos limpios a uber_trips_prod ---")
df_clean.write \
    .format("org.apache.spark.sql.cassandra") \
    .mode("append") \
    .options(table=TABLE_PROD, keyspace=KEYSPACE) \
    .save()

print(f"RESULTADO: {filas_depositadas} filas DEPOSITADAS en la tabla '{TABLE_PROD}'.")

print("\nDatos DEPOSITADOS correctamente en uber_trips, uber_trips_clean y uber_trips_prod.")



--- DEPOSITANDO datos limpios a uber_trips_clean ---


RESULTADO: 1000 filas DEPOSITADAS en la tabla 'uber_trips_clean'.
--- DEPOSITANDO datos limpios a uber_trips_prod ---


[Stage 30:=============================================>          (25 + 6) / 31]

RESULTADO: 1000 filas DEPOSITADAS en la tabla 'uber_trips_prod'.

Datos DEPOSITADOS correctamente en uber_trips, uber_trips_clean y uber_trips_prod.


In [8]:
# =======================================================
# 8. EXPORTACIÓN A CSV (Resultados Agregados)
# =======================================================

os.makedirs("output/clean_data", exist_ok=True)
os.makedirs("output/aggregated_data", exist_ok=True)

df_clean.write.mode("overwrite").csv("output/clean_data", header=True)
df_agg_uber_sales.write.mode("overwrite").csv("output/aggregated_data", header=True)
print("Datos limpios y agregados exportados a CSV.")


[Stage 34:=====================================>                 (21 + 10) / 31]

Datos limpios y agregados exportados a CSV.


### cierre de conexiòn

In [9]:
# =======================================================
# 9. CIERRE DE CONEXIONES
# =======================================================

session.shutdown()
cluster.shutdown()
spark.stop()
print("\n--- Proceso ETL Completado. Conexiones a Spark y Cassandra cerradas. ---")


--- Proceso ETL Completado. Conexiones a Spark y Cassandra cerradas. ---


<b>Configuración del productor en Kafka (Tabla cruda)</b>
<br>
Se establece la conexión con Cassandra para consultar la tabla `uber_trips`, almacenando los registros en una variable. Posteriormente, se importan las librerías necesarias para Kafka y JSON con el fin de serializar los datos y establecer la conexión con el servicio. Finalmente, se empaquetan las columnas requeridas en un diccionario y se envían al tópico `uber_trips` con un intervalo de un segundo entre cada transacción.

In [3]:
from cassandra.cluster import Cluster
from kafka import KafkaProducer
import json
import threading
import time
from datetime import datetime, timezone

# =======================================
# 1. CASSANDRA & KAFKA CONFIGURACIÓN
# =======================================

# --- AJUSTE CRÍTICO DE HOST ---
# Si tu ETL de Spark usó 'localhost' y estás ejecutando este script en la misma máquina,
# cambia '100.68.89.127' a 'localhost'.
CASSANDRA_HOST = '100.68.89.127' 
CASSANDRA_PORT = '9042'
KEYSPACE = 'trips'

try:
    cluster_src = Cluster([CASSANDRA_HOST], port=int(CASSANDRA_PORT))
    session_src = cluster_src.connect(KEYSPACE)
    print(f"Conexión con Cassandra establecida en el host: {CASSANDRA_HOST}")
except Exception as e:
    print(f"ERROR CRÍTICO: No se pudo conectar a Cassandra en {CASSANDRA_HOST}:{CASSANDRA_PORT}.")
    print("Asegúrate de que el servicio esté corriendo y la IP sea correcta.")
    raise e


# Configuración de Kafka (Destino de los eventos)
KAFKA_BROKER = '100.68.89.127:9092' 
producer = KafkaProducer(
    bootstrap_servers=KAFKA_BROKER,
    # Serializa el diccionario de Python a JSON y luego a bytes (utf-8)
    value_serializer=lambda m: json.dumps(m).encode('utf-8')
)
print(f"Conexión a Kafka Broker establecida: {KAFKA_BROKER}")

# Definición de la lista de columnas para el SELECT (mejora la eficiencia en Cassandra)
COLUMNAS_CASSANDRA_SELECT = """
    index_trip, request_datetime, pickup_datetime, dropoff_datetime, 
    pulocationid, dolocationid, base_passenger_fare, tolls, 
    bcf, sales_tax, congestion_surcharge, airport_fee, tips, 
    driver_pay, shared_request_flag, shared_match_flag, access_a_ride_flag,
    wav_request_flag, wav_match_flag, on_time_pickup, hour, year, month, day, 
    uber_sales, pickup_zone, delivery_zone
"""

# =======================================
# 2. PRODUCER 1 – uber_trips_clean
# =======================================
def produce_uber_trips_clean():
    """Lee datos de uber_trips_clean y los envía al topic 'uber_trips_clean'."""
    TOPIC = "uber_trips_clean"
    TABLE = "uber_trips_clean"
    print(f"\n-> Producer {TOPIC} iniciado")
    
    # Lectura de la tabla de datos limpios: limitamos a 50 para simulación.
    query = f"SELECT {COLUMNAS_CASSANDRA_SELECT} FROM {TABLE} LIMIT 50"
    
    try:
        rows = session_src.execute(query)
    except Exception as e:
        print(f"ERROR al ejecutar consulta CQL en {TABLE}: {e}")
        return

    has_data = False
    count = 0
    for row in rows:
        has_data = True
        count += 1
        
        msg = {
            "index_trip": row.index_trip,
            "request_datetime": row.request_datetime.replace(tzinfo=timezone.utc).isoformat(),
            "pickup_datetime": row.pickup_datetime.replace(tzinfo=timezone.utc).isoformat(),
            "dropoff_datetime": row.dropoff_datetime.replace(tzinfo=timezone.utc).isoformat(),
            "pulocationid": row.pulocationid, 
            "dolocationid": row.dolocationid,
            "base_passenger_fare": row.base_passenger_fare,
            "tips": row.tips,
            "driver_pay": row.driver_pay,
            "uber_sales": row.uber_sales,
            "pickup_zone": row.pickup_zone,
            "delivery_zone": row.delivery_zone,
            "source": TABLE,
            "sent_at": datetime.now(timezone.utc).isoformat()
        }

        producer.send(TOPIC, msg)
        print(f"-> {TOPIC}: index_trip {row.index_trip} enviado. Total: {count}")
        time.sleep(5) # Simulación de envío

    if not has_data:
        print(f"ADVERTENCIA: Tabla {TABLE} NO TIENE DATOS para enviar. (Revisa el ETL)")
    else:
        print(f"Completado: {TABLE}: {count} filas leídas y enviadas.")

# =======================================
# 3. PRODUCER 2 – uber_trips_prod
# =======================================
def produce_uber_trips_prod():
    """Lee datos de uber_trips_prod y los envía al topic 'uber_trips_prod'."""
    TOPIC = "uber_trips_prod"
    TABLE = "uber_trips_prod"
    print(f"\n-> Producer {TOPIC} iniciado")
    
    # Lectura de la tabla de producción: limitamos a 50 para simulación.
    query = f"SELECT {COLUMNAS_CASSANDRA_SELECT} FROM {TABLE} LIMIT 50"
    
    try:
        rows = session_src.execute(query)
    except Exception as e:
        print(f"ERROR al ejecutar consulta CQL en {TABLE}: {e}")
        return

    has_data = False
    count = 0
    for row in rows:
        has_data = True
        count += 1
        
        msg = {
            "index_trip": row.index_trip,
            "request_datetime": row.request_datetime.replace(tzinfo=timezone.utc).isoformat(),
            "pickup_datetime": row.pickup_datetime.replace(tzinfo=timezone.utc).isoformat(),
            "dropoff_datetime": row.dropoff_datetime.replace(tzinfo=timezone.utc).isoformat(),
            "pulocationid": row.pulocationid,
            "dolocationid": row.dolocationid,
            "base_passenger_fare": row.base_passenger_fare,
            "tips": row.tips,
            "driver_pay": row.driver_pay,
            "uber_sales": row.uber_sales,
            "pickup_zone": row.pickup_zone,
            "delivery_zone": row.delivery_zone,
            "source": TABLE,
            "sent_at": datetime.now(timezone.utc).isoformat()
        }

        producer.send(TOPIC, msg)
        print(f"-> {TOPIC}: index_trip {row.index_trip} enviado. Total: {count}")
        time.sleep(5) 

    if not has_data:
        print(f"ADVERTENCIA: Tabla {TABLE} NO TIENE DATOS para enviar. (Revisa el ETL)")
    else:
        print(f"Completado: {TABLE}: {count} filas leídas y enviadas.")


# =======================================
# 4. EJECUCIÓN SIMULTÁNEA Y CIERRE
# =======================================
if __name__ == "__main__":
    
    # Inicialización de hilos para ejecución concurrente
    t1 = threading.Thread(target=produce_uber_trips_clean)
    t2 = threading.Thread(target=produce_uber_trips_prod)

    t1.start()
    t2.start()

    # Esperar a que ambos hilos terminen
    t1.join()
    t2.join()

    # Asegurar que todos los mensajes pendientes sean enviados
    producer.flush()
    
    # Cerrar conexiones
    producer.close()
    session_src.shutdown()
    cluster_src.shutdown()

    print("\nProceso de producción finalizado. Conexiones a Kafka y Cassandra cerradas.")

Conexión con Cassandra establecida en el host: 100.68.89.127
Conexión a Kafka Broker establecida: 100.68.89.127:9092

-> Producer uber_trips_clean iniciado

-> Producer uber_trips_prod iniciado
-> uber_trips_clean: index_trip 769 enviado. Total: 1
-> uber_trips_prod: index_trip 769 enviado. Total: 1
-> uber_trips_clean: index_trip 23 enviado. Total: 2
-> uber_trips_prod: index_trip 23 enviado. Total: 2
-> uber_trips_prod: index_trip 114 enviado. Total: 3
-> uber_trips_clean: index_trip 114 enviado. Total: 3
-> uber_trips_prod: index_trip 660 enviado. Total: 4
-> uber_trips_clean: index_trip 660 enviado. Total: 4
-> uber_trips_prod: index_trip 893 enviado. Total: 5-> uber_trips_clean: index_trip 893 enviado. Total: 5

-> uber_trips_prod: index_trip 53 enviado. Total: 6
-> uber_trips_clean: index_trip 53 enviado. Total: 6
-> uber_trips_prod: index_trip 987 enviado. Total: 7
-> uber_trips_clean: index_trip 987 enviado. Total: 7
-> uber_trips_prod: index_trip 878 enviado. Total: 8
-> uber_